In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql
import ast
import requests
import random

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
def get_game_id():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [4]:
# 获取近期所有计划
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                * 
            FROM
                db_ptom.ptom_third_plan p
            WHERE
                game_id IN ({})
                AND media_id = 45
                AND opt_status = 1
                AND create_time>=date( NOW() - INTERVAL 1440 HOUR )
                AND create_time<= date(NOW())
                            AND plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= date( NOW() - INTERVAL 1440 HOUR )
                                and create_time <= date( NOW() - INTERVAL 1416 HOUR )
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

In [5]:
# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND a.create_time >= date( NOW() - INTERVAL 2880 HOUR ) 
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    result_df = result_df.dropna(axis=0, how='any')
    result_df['channel_id'] = result_df['channel_id'].map(int)
    result_df['source_id'] = result_df['source_id'].map(int)
    result_df['image_id'] = result_df['image_id'].map(int)
    cur.close()
    conn.close()
    return result_df

In [6]:
def get_launch_report():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount as roi,
            b.pay_role_user_num / b.create_role_num as pay_rate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 45
            AND b.game_id IN ({})
            AND b.amount >= 500
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df

In [7]:
# 获取近期优化计划的创意数据
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            a.chl_user_id 'channel_id',
            a.source_id,
            a.image_id,
            b.creative_info 
        FROM
            (
            SELECT
                a.chl_user_id,
                a.source_id,
                a.image_id,
                b.ad_id,
                b.ad_account_id 
            FROM
                db_ptom.ptom_plan a
                LEFT JOIN db_ptom.ptom_third_plan b ON a.plan_id = b.plan_id 
            WHERE
                a.media_id = 45 
                AND a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
                AND a.game_id IN ({}) 
                AND b.ad_id IS NOT NULL 
            ) a
            INNER JOIN db_ptom.ptom_third_ad_creative b ON a.ad_id = b.ad_id 
            AND a.ad_account_id = b.ad_account_id 
        WHERE
            b.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.media_id = 45 
            AND a.image_id IS NOT NULL
            

    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result

In [8]:
# 获取近期优化计划的创意数据
def get_document():
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            pd.doc_content
        FROM
            db_ptom.ptom_document_creative_relation pdcr
            INNER JOIN db_ptom.ptom_plan_document pd ON pd.doc_id = pdcr.doc_id 
        WHERE
            pdcr.create_time >= '2020-01-01' and pdcr.media_id in (10,16,45,32) 
            AND pd.game_ids IS NOT NULL 
            AND pd.game_ids != '' 
            AND FIND_IN_SET(1001379, pd.game_ids)
            GROUP BY
                pdcr.doc_id 
            ORDER BY
                count(*) DESC 
                LIMIT 20
    '''
    cur.execute(sql)
    result = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result['doc_content'].values

In [9]:
def get_plan_json(plan_info):
    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)

    temp = plan_info['audience'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('audience', axis=1, inplace=True)

    temp = plan_info['ocpc'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ocpc', axis=1, inplace=True)

    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'create_time',
           'bidtype', 'producttypes', 'ftypes', 'pause', 'autoExpansion',
           'education', 'customAge', 'keywords', 'keywordsExtend', 'newInterests',
           'sex', 'iosVersion', 'androidVersion', 'excludeTrans', 'region',
           'device', 'age', 'transType', 'ocpcLevel', 'isSkipStageOne', 'payMode',
           'optimizeDeepTrans', 'transFrom', 'ocpcBid', 'deepOcpcBid',
           'deepTransType', 'roiRatio', 'useRoi']]
    
    # plan_info 去重
    plan_info = plan_info.sort_values(by='create_time')
    plan_info = plan_info.drop_duplicates(subset=['channel_id','source_id'], keep='last')
    return plan_info

In [23]:
plan_info = get_plan_info()
plan_info = get_plan_json(plan_info)

In [25]:
plan_info['keywords'].value_counts()

1,1.70月卡传奇,1.76传奇复古,1.76传奇手游官网,1.76传奇新服,1.76暗黑传奇,1.76毁灭版本,1.76的手游传奇,1.76神龙毁灭版本,1.76经典传奇手游,1.76老复古,1.85传奇手游,1.85火龙传奇手游,1.85火龙版传奇手游,1.85火龙版本传奇,100000000迷你币永久激活码,1000炮打鱼游戏下载,1000炮捕鱼机,10倍攻速传奇手游,17173官网,17175捕鱼达人,173游戏交易平台,175传奇手游,176传奇,176复古传奇小极品,180传奇,180复古火龙,185传奇,196黄金皓月传奇,2000左右的电脑推荐,200斤如何减肥,2017传奇手游哪个好玩,2018仙侠网游,2018奇迹mu,2018年dnf手游官网,2019开奖结果开奖,2020传奇手游,20万左右买什么车好,20万轻钢别墅图片,20元红包,2d手游,350手游烈火龙城下载,37手游网,3975复古传奇官网,3975复古传奇手游辅助,3975复古传奇礼包,3d版传奇手游有哪些,4399游戏盒,4s手机,69美女直播,76复古传奇手游,76版本传奇,80复古传奇,888捕鱼,888棋牌游戏大厅,921传奇3,92捕鱼,9377,9377传奇游戏,9游游戏中心,app下载,app下载软件下载,arpg手游排行,bt手游大全,bt游戏平台,cba,cf,cf手游,dnf不知火,dnf修罗,dnf勇士之路,dnf地下城与勇士sf,dnf地下城与勇士刷图卡,dnf地下城与勇士单,dnf官方网站,dnf官网,dnf手游版,dnf新职业,dnf武神,dnf版本,dnf狂战士,dnf礼包,dnf阿修罗,dnf魂链,dnf魔神,dota2,fgo,gm版传奇手游平台,ios,ios和安卓互通传奇手游,iphone 8,jeep,jj千炮捕鱼刷金币教程,lol,lpl,mmorpg回合制游戏,mmorpg手游排行榜,moba手游排行榜,nba,nba总冠军赛,nba总决赛高清录像,nba最强球员,nba球星,nga魔兽世界,ooxx漫画大全,oppo,oppo手机好不好,p7华为,ps4,qq,qq下载,qq情侣头像,qq情侣头像图片,qq手游礼包领取中心,reebok复古classic,rpg,switch,txt全本免费小说缓存,txt全本小说,vivo手机排行,

In [24]:
plan_info.head()

,ad_account_id,game_id,channel_id,source_id,create_time,bidtype,producttypes,ftypes,pause,autoExpansion,education,customAge,keywords,keywordsExtend,newInterests,sex,iosVersion,androidVersion,excludeTrans,region,device,age,transType,ocpcLevel,isSkipStageOne,payMode,optimizeDeepTrans,transFrom,ocpcBid,deepOcpcBid,deepTransType,roiRatio,useRoi
0,11409,1001446,20143,1425722,2022-05-06 11:55:33,3,[],"[1, 2, 8]",False,0,0,0,".传奇,03年传奇版本,1.70版本传奇,1.75版传奇,1.76 冰雪传奇,1.76传奇单...",1,0,2,0,0,0,9999999,1,"2,3,4,5",4,2,True,1,True,1,150.0,5000.0,26.0,NaN,NaN
1,11409,1001446,20143,1425723,2022-05-06 11:55:33,3,[],"[1, 2, 8]",False,0,0,0,".传奇,03年传奇版本,1.70版本传奇,1.75版传奇,1.76 冰雪传奇,1.76传奇单...",1,0,2,0,0,0,9999999,1,"2,3,4,5",4,2,True,1,True,1,160.0,5500.0,26.0,NaN,NaN
2,11409,1001446,20143,1425724,2022-05-06 11:55:33,3,[],"[1, 2, 8]",False,0,0,0,".传奇,03年传奇版本,1.70版本传奇,1.75版传奇,1.76 冰雪传奇,1.76传奇单...",1,0,2,0,0,0,9999999,1,"2,3,4,5",4,2,True,1,True,1,160.0,5500.0,26.0,NaN,NaN
3,11409,1001446,20143,1425725,2022-05-06 11:55:33,3,[],"[1, 2, 8]",False,0,0,0,".传奇,03年传奇版本,1.70版本传奇,1.75版传奇,1.76 冰雪传奇,1.76传奇单...",1,0,2,0,0,0,9999999,1,"2,3,4,5",4,2,True,1,True,1,156.0,5500.0,26.0,NaN,NaN
4,11409,1001446,20143,1425726,2022-05-06 11:55:33,3,[],"[1, 2, 8]",False,0,0,0,".传奇,03年传奇版本,1.70版本传奇,1.75版传奇,1.76 冰雪传奇,1.76传奇单...",1,0,2,0,0,0,9999999,1,"2,3,4,5",4,2,True,1,True,1,158.0,5500.0,26.0,NaN,NaN


In [10]:
def get_train_df():
    plan_info = get_plan_info()
    plan_info = get_plan_json(plan_info)
#     creative_info = get_creative()
#     creative_info = creative_info.drop_duplicates(subset=['channel_id','source_id'], keep='last')
#     creative_info['creative_info'] = creative_info['creative_info'].apply(json.loads)
#     temp = creative_info['creative_info'].apply(pd.Series)
#     if 'image_id' in temp.columns:
#         temp = temp.drop('image_id', axis=1)
#     temp = temp[['materialstyle','material']]
#     temp['material'] = temp['material'].apply(json.loads)
#     creative_info = pd.concat([creative_info,temp], axis=1)
#     creative_info.drop('creative_info', axis=1, inplace=True)
#     temp = creative_info['material'].apply(pd.Series)
#     temp = temp[['brand','elements','title','subtitle']]
#     creative_info = pd.concat([creative_info,temp], axis=1)
#     creative_info.drop('material', axis=1, inplace=True)
#     creative_info_1 = creative_info[creative_info['elements'].isna()]
#     creative_info_2 = creative_info[creative_info['elements'].notna()]
#     if creative_info_1.shape[0] != 0:
#         creative_info_1['titles'] = creative_info_1['title'].apply(lambda x: [x])
#     creative_info_2['titles'] = creative_info_2['elements'].apply(lambda x: x['titles'])
#     creative_info = pd.concat([creative_info_1,creative_info_2],axis=0)
#     creative_info.drop(['elements','title'],axis=1,inplace=True)
#     creative_info['subtitle'] = creative_info['subtitle'].fillna(creative_info['subtitle'].mode()[0])

#     plan_info = pd.merge(plan_info, creative_info, on=['channel_id','source_id'], how='inner')
    image_info = get_image_info()
    launch_report = get_launch_report()
    
    plan_info['create_time'] = pd.to_datetime(plan_info['create_time'])
    plan_info_current = plan_info[plan_info['create_time']>=pd.datetime.now() - pd.DateOffset(30)]
    image_info = image_info[image_info['image_id'].notna()]
    image_info['image_id'] = image_info['image_id'].astype(int)
    
    score_image_1 = get_score_image_other()
    score_image_2 = get_score_image()
    score_image = np.union1d(score_image_1,score_image_2)
    score_image = filter_game_image(score_image, 1001379)
    
    now_plan_roi = get_now_plan_roi()
#     df_create = pd.merge(plan_info_current,image_info,on=['channel_id','source_id','image_id'],how='left')
    df_create = pd.merge(plan_info_current,now_plan_roi.drop(['ad_account_id'], axis=1),on=['channel_id','source_id'],how='inner')
    # df_create['platform'] = df_create['platform'].astype(str)
    # df_create['platform'] = df_create['platform'].map({"['ANDROID']":1,"['IOS']":2})
    # df_create['platform'] = df_create['platform'].astype(int)
    df_create.to_csv('./df_create.csv',index=0)
    # 只跑付费ROI，不跑激活付费
#     df_create = df_create[df_create['optimizeDeepTrans']==False] 
    plan_create = create_plan(df_create,score_image)
    
    return plan_create

In [11]:
# 获取近期计划的运营数据
def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
            a.ad_account_id,
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_data_ptom.ptom_plan a 
        left join
            db_stdata.st_lauch_report b
        on a.chl_user_id=b.channel_id and a.source_id=b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 240 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 45 
            AND b.game_id IN ({}) 
            AND b.amount >= 200 
            AND b.platform = 2
            AND b.pay_role_user_num >= 1 
            AND b.new_role_money >= 6
            AND (b.new_role_money / b.amount)>=0.005
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
#     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

In [12]:
# 获取score_image (分数大于550的image_id)
def get_score_image():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
            SELECT
            image_id,
            label_ids 
        FROM
            dws.dws_image_score_d 
        WHERE
            dt = CURRENT_DATE 
            AND media_id = 45 
            AND image_create_role_roi >= 0.005 
            AND ( image_create_role_retain_1d >= 0.07 OR image_create_role_retain_1d IS NULL ) 
            AND score >= 500 
        GROUP BY
            image_id,
            label_ids
    '''
    
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [13]:
def get_score_image_other():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
        SELECT
            image_id,
            label_ids 
        FROM
            dws.dws_image_score_d 
        WHERE
            dt = CURRENT_DATE 
            AND media_id in (10,16,32) 
            AND image_create_role_roi >= 0.008 
            AND ( image_create_role_retain_1d >= 0.07 OR image_create_role_retain_1d IS NULL ) 
            AND score >= 550 
        GROUP BY
            image_id,
            label_ids
    '''
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [14]:
def filter_exclude_image(image_ids, media_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND NOT FIND_IN_SET({}, exclude_media_ids)
    '''
    result = pd.read_sql(sql.format(image_ids, media_id), conn)
    # result = result[result['image_name'].apply(lambda x:True if len(re.findall(r"QMZZ(.*?)-", x))>0 and int(re.findall(r"QMZZ(.*?)-", x)[0]) not in tt_ban else False)]
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [15]:
def filter_game_image(image_ids, game_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND FIND_IN_SET({}, game_ids)
    '''
    cur.execute(sql.format(image_ids, game_id))
    result = pd.read_sql(sql.format(image_ids, game_id), conn)

    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [16]:
# 303机器人勿上
def get_lable_image():
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            image_id 
        FROM
            db_ptom.ptom_image_info a 
        WHERE
            FIND_IN_SET( 303, label_ids ) 

    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df['image_id'].values

In [17]:
# 对长期累积roi不达标的素材进行过滤
# 获取数据-1-6天
def get_data_1_6():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
            SELECT
                user_id,
                game_id,
                channel_id,
                source_id,
                role_id,
                platform,
                media_id,
                pay_num,
                pay_sum,
                create_role_time,
                created_role_day,
                pay_7_pred,
                dt 
            FROM
                tmp_data.tmp_roles_portrait_info_predict 
            WHERE
                dt = CURRENT_DATE
                AND channel_id = 21206
                AND game_id IN ({})
    
    '''
    finalSql = sql.format(game_id)
    cursor.execute(finalSql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result

def get_data_7():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
        SELECT
            user_id,
            game_id,
            channel_id,
            source_id,
            role_id,
            platform,
            media_id,
            pay_num,
            pay_sum,
            create_role_time,
            created_role_day,
            pay_sum AS pay_7_pred,
            dt 
        FROM
            tmp_data.tmp_roles_portrait_info_train2 
        WHERE
            dt = CURRENT_DATE 
            AND created_role_day = 7
            AND channel_id = 21206
            AND game_id IN ({})
    '''
    finalSql = sql.format(game_id)
    cursor.execute(finalSql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result

# 获取近期所有计划的消耗情况
def get_amount_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
            SELECT
                tdate,
                channel_id,
                source_id,
                game_id,
                media_id,
                platform,
                amount 
            FROM
                db_stdata.st_lauch_report 
            WHERE
                game_id IN ({}) 
                AND tdate_type = 'day' 
                AND channel_id = 21206
                AND tdate >= date( NOW() - INTERVAL 7 DAY )
                AND tdate <= date( NOW() - INTERVAL 1 DAY )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    
    return result_df

def calculate_roi(groupby_list=['media_id','platform','image_id']):
    # 获取预测数据
    df_1 = get_data_7()
    df_2 = get_data_1_6()
    df = df_1.append(df_2)
    df['create_role_time'] = pd.to_datetime(df['create_role_time']).dt.date
    # 聚合生成计划维度的回款预测数据
    source_df = pd.DataFrame({'7_pay_sum':df.groupby(['game_id','channel_id','source_id','create_role_time','media_id','platform'])['pay_7_pred'].sum()}).reset_index()
    source_df['media_id'] = source_df['media_id'].map(int)
    source_df = source_df[source_df['media_id'].isin([10,16,32,45])]
    # 获取计划消耗数据，只取4个主流媒体
    amount_info = get_amount_info()
    amount_info = amount_info[amount_info['media_id'].isin([10,16,32,45])]
    image_info = get_image_info()
    # 拼接数据
    source_df.rename(columns={'create_role_time':'tdate'}, inplace=True)
    source_df['tdate'] = pd.to_datetime(source_df['tdate'])
    source_df = pd.merge(source_df,amount_info,on=['channel_id','source_id','tdate','media_id','platform','game_id'],how='outer')
    source_df['amount'] = source_df['amount'].fillna(0)
    source_df['7_pay_sum'] = source_df['7_pay_sum'].fillna(0)
    data = pd.merge(source_df,image_info,on=['channel_id','source_id'],how='left')
    
    df_pay_sum = pd.DataFrame({'7_pay_sum': data.groupby(groupby_list)['7_pay_sum'].sum()}).reset_index()
    df_amount = pd.DataFrame({'7_amount': data.groupby(groupby_list)['amount'].sum()}).reset_index()
    df = pd.merge(df_amount,df_pay_sum,on=groupby_list,how='left')
    df['7_pay_sum'] = df['7_pay_sum'].apply(lambda x: round(x,2))
    df['7_amount'] = df['7_amount'].apply(lambda x: round(x,2))
    df = df[df['7_amount']>0]
    df['7_roi'] = round(df['7_pay_sum'] / df['7_amount'],4)
    
    df = df[(df['media_id']==45)&(df['platform']==2)&(df['7_amount']>=15000)&(df['7_roi']<0.05)]
    df['image_id'] = df['image_id'].astype(int)
    return df['image_id'].values

In [18]:
# calculate_roi()

In [19]:
def create_plan(df_create,score_image):
    # 选ad_account_id、image_id每个账号+素材8条
    game_id = 1001446  ## 选择包：
#     df_create = df_create[df_create['game_id'].isin([1001794, 1001862, 1001783, 1001863, 1001961, 1001935])]
#     image_id_group = np.intersect1d(df_create['image_id'].unique(), score_image)
    image_id_group = filter_exclude_image(score_image, 45)
    image_id_group = np.setdiff1d(image_id_group,calculate_roi())
    image_id_group = np.setdiff1d(image_id_group, get_lable_image())  # 过滤指定标签的素材
    print('匹配的素材ID', image_id_group)
#     df_create = df_create[df_create['image_id'].isin(image_id_group)]
    
    # 选择预算，不限制预算
    plan = pd.DataFrame({'image_id': image_id_group})
    # 'materialstyle','brand', 'titles', 'subtitle'
    sample_df = df_create[['producttypes', 'ftypes','education', 'customAge', 'keywords', 'keywordsExtend', 'newInterests',
                       'sex', 'iosVersion', 'androidVersion', 'excludeTrans', 'region',
                       'device', 'age','transType', 'ocpcLevel', 'isSkipStageOne', 'payMode',
                   'optimizeDeepTrans', 'transFrom', 'ocpcBid', 'deepOcpcBid',
                   'deepTransType', 'roiRatio', 'useRoi']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    plan = plan.reset_index(drop=True)
    plan['game_id'] = game_id
    return plan

In [20]:
def get_plan(plan_num, ad_account_id_group):
    plan_create = get_train_df()  
    # 每个账号分配计划数量
    plan_result = pd.DataFrame()
    for ad_account_id in ad_account_id_group:
        if plan_create.shape[0] < plan_num:
            plan_num = plan_create.shape[0]
        plan_result_ = plan_create.sample(n=plan_num, replace=False).reset_index(drop=True)
    #     plan_create = plan_create.drop(plan_create[plan_create['image_id'].isin(plan_result_.image_id.values)].index, axis=0)
        plan_result_['ad_account_id'] = ad_account_id
        plan_result = plan_result.append(plan_result_)   
    plan_result['autoExpansion'] = 0
    plan_result['pause'] = False
    
    plan_result = plan_result.reset_index(drop=True)
    plan_result['deepTransTypes'] = plan_result.apply(lambda x: [26],axis=1)   # 转化参数
    
    plan_result['plan_auto_task_id'] = "12550,12623"
#     plan_result['plan_auto_task_id'] = "12621"
    plan_result['op_id'] = 13268
    plan_result['subject'] = 2   # 跑IOS
    plan_result['flag'] = 'BD'
    plan_result['game_name'] = '曙光'
    
    # 定向
    plan_result['customAge'] = np.nan
    plan_result['keywords'] = plan_result['keywords'].apply(lambda x: "1,1.70月卡传奇,1.76传奇复古,1.76传奇手游官网,1.76传奇新服,1.76暗黑传奇,1.76毁灭版本,1.76的手游传奇,1.76神龙毁灭版本,1.76经典传奇手游,1.76老复古,1.85传奇手游,1.85火龙传奇手游,1.85火龙版传奇手游,1.85火龙版本传奇,100000000迷你币永久激活码,1000炮打鱼游戏下载,1000炮捕鱼机,10倍攻速传奇手游,17173官网,17175捕鱼达人,173游戏交易平台,175传奇手游,176传奇,176复古传奇小极品,180传奇,180复古火龙,185传奇,196黄金皓月传奇,2000左右的电脑推荐,200斤如何减肥,2017传奇手游哪个好玩,2018仙侠网游,2018奇迹mu,2018年dnf手游官网,2019开奖结果开奖,2020传奇手游,20万左右买什么车好,20万轻钢别墅图片,20元红包,2d手游,350手游烈火龙城下载,37手游网,3975复古传奇官网,3975复古传奇手游辅助,3975复古传奇礼包,3d版传奇手游有哪些,4399游戏盒,4s手机,69美女直播,76复古传奇手游,76版本传奇,80复古传奇,888捕鱼,888棋牌游戏大厅,921传奇3,92捕鱼,9377,9377传奇游戏,9游游戏中心,app下载,app下载软件下载,arpg手游排行,bt手游大全,bt游戏平台,cba,cf,cf手游,dnf不知火,dnf修罗,dnf勇士之路,dnf地下城与勇士sf,dnf地下城与勇士刷图卡,dnf地下城与勇士单,dnf官方网站,dnf官网,dnf手游版,dnf新职业,dnf武神,dnf版本,dnf狂战士,dnf礼包,dnf阿修罗,dnf魂链,dnf魔神,dota2,fgo,gm版传奇手游平台,ios,ios和安卓互通传奇手游,iphone 8,jeep,jj千炮捕鱼刷金币教程,lol,lpl,mmorpg回合制游戏,mmorpg手游排行榜,moba手游排行榜,nba,nba总冠军赛,nba总决赛高清录像,nba最强球员,nba球星,nga魔兽世界,ooxx漫画大全,oppo,oppo手机好不好,p7华为,ps4,qq,qq下载,qq情侣头像,qq情侣头像图片,qq手游礼包领取中心,reebok复古classic,rpg,switch,txt全本免费小说缓存,txt全本小说,vivo手机排行,xxoo29免播放器电影,yy影院,yy直播美女主播,yy绝地求生开黑频道,一,一世之尊,一刀999,一刀9999999亿传奇破解版,一刀999级传奇,一刀传世游戏,一夜倩找附近的女人,一念成瘾，陆太太嫁到,一美女,万世传奇,万古天帝,万古神王,万古至尊,万域之王,万炮捕鱼达人,万科城,万科城房价,三国,三国之吕布天下,三国传奇,三国名将,三国城池游戏,三国志礼包,三国战争类游戏,三国战记游戏,三国战记网游,三国放置版,三国新吕布,三国游戏手机版,三国第一猛将并非吕布,三国类手游卡牌,三国群英传,三国英杰传2,三寸人间,三星,三端互通传奇开服列表,三端互通单职业打金传奇,三职业传奇手游,上古异兽录,上古神魔,上海,上线送vip15的传奇手游,上线送满级vip传奇,下载app,下载传奇,下载手机壁纸,下载手机游戏安卓,下载捕鱼捕鱼,下载游戏大厅,下载美女直播秀,不挂机的传奇手游,不灭帝尊,不花钱的回合制手游,不花钱的手游排行榜,不花钱的游戏,不要网的手机游戏,不败战神,不锈钢切割机,世界,世界八大奇迹,东方不败,两人打麻将,丧尸出笼,丧尸围城电影,丧尸来了,丧尸生存游戏手游,个人写真,中国游戏交易网,中国足球,中国黄金什么价格,中心游戏中心,丰满,丰田价格,丰田普拉多报价,丰田汉兰达二手车价格,主宰三界,么么直播,九天战神,九星霸体诀,九游,九阳帝尊,九阴真经,买iphone手机,买三星手机,买华为手机,买苹果手机,乱神诀,乾坤,乾坤传奇,事件,二战风云,二手房交易市场,二手房价格,二手房出售,二手房房价,二手本田crv,五月天,亚洲,人体艺术,人多的传奇手游,人气仙侠手游,人道至尊,亿炮捕鱼,什么传奇不花钱,什么传奇好玩,什么传奇爆率最高,什么传奇爆率高,什么手游可以赚人民币,什么手游好玩不花钱,什么手游能赚钱,什么手游装备全靠爆,仙侠传奇,仙侠修仙世界手游,仙侠大陆手游,仙侠御剑修仙,仙侠手游游戏排行榜前十名,仙侠游戏名字,仙侠礼包,仙侠网游名字,仙侠问道下载,仙侠问道游戏手游,仙侠闯关游戏,仙凡幻想公测,仙剑,仙剑一,仙剑奇侠传,仙剑情缘版本传奇,仙武帝尊,仙灵世界,仙落凡尘,仙道,代练王者荣耀,伏天氏,传世刀刀爆,传奇,传奇 单职业,传奇1.10版本,传奇1.76,传奇1.76升级武器,传奇1.76单机版,传奇1.76砸武器攻略,传奇1.76超爆版无限金币游戏,传奇1.76金币服,传奇1.76金币版本,传奇170金币版手游,传奇176版,传奇176版本,传奇180版本,传奇180金币,传奇2单机版,传奇3手游游戏,传奇3有手游吗,传奇3武器,传奇3盛世,传奇4游戏,传奇4职业,传奇999级,传奇s,传奇一刀,传奇一刀999级,传奇一区,传奇一条龙,传奇三,传奇上线10000级,传奇世界,传奇世界2手机版,传奇世界之不死之身,传奇世界传奇永恒,传奇世界充值,传奇世界单机版,传奇世界带元神,传奇世界手游,传奇世界手游怪物爆率,传奇世界打金服,传奇世界新手卡,传奇世界新装备,传奇之刃,传奇之纵横玛法,传奇交易平台,传奇任务版,传奇任达华,传奇信息网,传奇充值,传奇免费,传奇内挂,传奇冰雪服,传奇冰雪版,传奇冰雪版本,传奇刀刀光柱,传奇刀刀切割,传奇刀刀暴击版,传奇单机版,传奇单机版游戏,传奇单职业,传奇单职业冰雪版本手游,传奇单职业打金,传奇单职业超变手游,传奇单职业超变版手游网站,传奇单职业高爆版,传奇变态版本,传奇变态版本下载,传奇变态版网页,传奇合击,传奇合击手游排行榜,传奇合击版,传奇和传奇2,传奇哪个版本最好玩,传奇哪个网站,传奇回归,传奇塔防,传奇复古1.76手游排行榜,传奇复古版本,传奇复古版游戏,传奇大全,传奇大全网站,传奇大哥,传奇威力,传奇子聪,传奇官方网站,传奇官网,传奇序章,传奇开区,传奇开区一条龙,传奇开区网,传奇开服网,传奇怀旧,传奇怀旧版手游官网,传奇怀旧版本,传奇怎么打怪,传奇怎么样,传奇怒斩,传奇怒火一刀,传奇战域,传奇战神,传奇手机版,传奇手机版下载,传奇手机电脑互通版,传奇手游,传奇手游1.76复古小极品,传奇手游1.85,传奇手游2,传奇手游bt版,传奇手游bt版本,传奇手游npc在哪里,传奇手游下载,传奇手游代理,传奇手游冰雪传奇,传奇手游决战沙城,传奇手游制作,传奇手游刷怪时间,传奇手游加速,传奇手游升级,传奇手游单机版,传奇手游合成版本大全排行榜,传奇手游哪个好玩,传奇手游哪个正版,传奇手游复古传奇,传奇手游多少级可以强化,传奇手游好玩吗,传奇手游官方网,传奇手游官方网站,传奇手游开服,传奇手游开服时间,传奇手游开服网,传奇手游开服网站,传奇手游怀旧版,传奇手游找服网站,传奇手游技能点怎么用,传奇手游挂机,传奇手游挂机地图,传奇手游排行榜,传奇手游排行榜 人气 第一名 经典,传奇手游新版本,传奇手游无限元宝,传奇手游最快升级方法,传奇手游有哪些,传奇手游未知暗殿,传奇手游正版,传奇手游深渊,传奇手游游戏平台,传奇手游游戏网站,传奇手游火龙版本,传奇手游点卡版,传奇手游版,传奇手游王者战神,传奇手游破解,传奇手游神途,传奇手游福利版,传奇手游纯公益服,传奇手游网,传奇手游装备在哪打,传奇手游账号交易平台,传奇手游辅助,传奇打装备,传奇打装备手游,传奇打金手游排行榜,传奇打金服手游,传奇打金服排行,传奇挂机,传奇挂机下载,传奇挂机游戏,传奇散人手游,传奇散人服,传奇散人版,传奇新服,传奇新版,传奇无双ol,传奇无双手游,传奇无双装备,传奇无限内购破解版,传奇暴击版,传奇有啥好玩的,传奇有手游,传奇有没有手游,传奇服,传奇服务端,传奇服网,传奇本版,传奇来了满v无限元宝,传奇武器,传奇武器大全,传奇永恒免费吗,传奇永恒怎么样,传奇游,传奇游戏名字大全,传奇游戏大全,传奇游戏手游1.76,传奇游戏排行榜,传奇满v无限元宝服,传奇火龙,传奇热血官网,传奇版本下载基地,传奇版本库,传奇版本手机游戏,传奇王者归来,传奇王者游戏,传奇现在还有吗,传奇登陆器下载,传奇百区,传奇的手机游戏,传奇的手游,传奇的网站,传奇盒子,传奇神兽,传奇私,传奇类手机游戏,传奇类手游,传奇类手游哪个不坑钱,传奇类手游挂机版,传奇类手游排行榜,传奇类的手游,传奇类的手游下载,传奇级,传奇续章,传奇至尊,传奇英雄游戏,传奇英雄版本,传奇蓝月手游,传奇角色,传奇论坛,传奇超变,传奇超变,刀刀切割,传奇超变单职业,传奇超变版,传奇超变版本下载,传奇超变高爆版破解版,传奇超爆版手游,传奇超级变,传奇超级爆率版,传奇超级爆率版手游,传奇达叔,传奇迷失,传奇迷失传奇,传奇迷失版本,传奇道士加点,传奇霸业游戏,传奇霸业激活码,传奇霸主,传奇高爆版单职业,传奇高爆率,传说,伦理片,体育,你好,使命召唤,侠客行,侠客行手游,修仙传,修仙劫,修仙劫在线阅读,修仙啊,修仙文,修仙记,修改器,修真之都市重生小说,修罗武神全本免费阅读,倚天屠龙记,傲视战神,僵尸战争,僵尸游戏大全,元宝,元气骑士破解版,光柱传奇,光通版传奇3,免费下载游戏,免费交友网站,免费全本小说九星霸体决,免费在线观看,免费小说修仙狂徒,免费小说哪里看,免费小说大全下载,免费小说小说下载txt,免费小说阅读器,免费已完结全本小说,免费爱看小说软件,免费玩游戏,免费的传奇,免费的手游,免费高清看电影,免费高清视频在线观看,全免费小说阅读,全国未来三天天气预报,全小说,全文免费阅读,全新奇迹手游,全新版本传奇,全本免费小说都市修仙,全民奇迹强化15漏洞,全民奇迹手游,全民奇迹攻略,全民奇迹无限钻石,全民奇迹游戏,全靠打怪爆装备的手游,六道仙尊,共和国之辉红警,关于三国的手游,关于二次元,兽王,兽破苍穹,军事战役,冬天钓鱼鲫鱼饵料,冬季钓鲫鱼饵料配方,冰雪传奇,冰雪传奇单职业打金,冰雪传奇官方,冰雪传奇手机版,冰雪传奇游戏,冰雪复古传奇手游官网,冰雪手游传奇,冰雪版本传奇打金手游,冰雪高爆版传奇手游,决战,决战沙场,决战皇城,减肥,凡人修仙传,凡人修真2元神,凡仙,刀光传奇手游,刀刀光柱传奇手游,刀刀切割传奇手游,刀刀暴击,刀刀烈火,刀剑,刀塔传奇2,刀塔传奇充值,刀塔传奇剑圣,刀塔传奇吧,刀塔传奇是什么类型的游戏,刀塔传奇游戏,刀塔传奇阵容,刘亦菲,刚开传奇,刚开传奇手游,别克gl8,别墅,刺激战场,剑三,剑与家园,剑侠情缘3网游,剑侠情缘冲值,剑侠江湖,剑啸江湖,剑圣,剑尊,剑灵传奇手游,剑网,剑魂,剑魔,加点,加盟连锁,动态壁纸,动漫之家,动物头像简笔画大全,动画手游,勇士队,勇者传说,化妆,北京,十万炮捕鱼游戏,千炮捕鱼2下载,千炮捕鱼官方网站,升级只靠打怪的传奇网游,升级游戏,华为,华为m3,华为mate,华为mate8,华为nova,华为p系列和mate系列哪个好,华为新款手机,华为路由器,华人捕鱼下载,单机传奇手游,单机传奇无需联网,单机游戏三国志2,单机版传奇游戏,单机版传奇类游戏下载,单机版手游传奇,单机麻将,单职业,单职业传奇bt手游版,单职业传奇吧,单职业传奇开服,单职业传奇手游刀刀切割,单职业传奇手游切割,单职业传奇手游排行榜,单职业传奇打金服,单职业传奇网站,单职业攻速传奇手游,单角色传奇,单身交友群,单身在线,单身女人电话,单身女人网,南通,厉害,原始176传奇,原始传奇,原始传奇复古版,原始传奇手游下载,原始传奇是什么,原始传奇页游,去哪儿旅行,受欢迎角色扮演游戏排行,变态传奇单职业,变态传奇单职业攻速,口袋妖怪3ds,口袋妖怪皮卡丘,口袋海贼王,口袋的妖怪,古墓传奇,古墓经典传奇,古天乐,古天乐贪玩蓝月,古驰,可以交易装备的手游,可以挂机手游,可以看美女的直播,史上第一祖师爷,叶罗丽,吃鸡,各种壁纸手机版,合击,合金装备,同城交友找情人,同城异性约会,名表品牌排名及价格,后院三国,吞噬星空,吞天诀,听有声小说,和传奇差不多的单机游戏,和地图,和女生聊天找话题,和平,和王者荣耀差不多的游戏,品牌加盟店,哈弗h2最新价格,哪个传奇手游好玩,哪个手游可以赚人民币,哪里买车便宜,唯美仙侠手游,啪啪三国2,噬魂,四合院,回到明朝当王爷,回合制网游大全,围城,国台酒酱香型白酒,国外买房,国战攻略,图片头像,土城争霸,圣墟免费阅读全文,圣斗士,圣斗士冥界,圣斗士手游,圣斗士新版,圣斗士星矢,圣斗士星矢游戏,圣斗士游戏,圣道传奇,在直播在线直播,在线捕鱼棋牌,在线观看,地下城与勇士,地下城与勇士啊,地下城与勇士手游,地下城与勇士手游国服,地下城与勇士手游版,地下城与勇士狂战士用什么武器,地图查询,地牢猎手,坦克世界手游,坦克世界闪击战,坦克前线,坦克前线二战,坦克风暴,塔防三国志庞德属性,塔防三国志黄色武将,塔防三国鲁肃,塔防策略游戏排行,塞尔达,复古传奇,复古传奇1,复古传奇1.76手游下载,复古传奇1.80,复古传奇3,复古传奇单职业,复古传奇去哪打装备,复古传奇开服,复古传奇开服时间表,复古传奇武器去那打,复古传奇祖玛阁走法,复古传奇网站,复古传奇装备爆率,复古传奇道士攻略,复古类传奇,复古绿色传奇,大三国,大众,大众新suv,大众新款suv,大众途观l报价,大劫主,大唐,大圣传,大型rpg手游,大型手游游戏,大天使之刃,大天使之剑吧,大天使之剑技巧,大天使之剑无尽之塔,大天使之剑礼包,大天使之剑魔剑士攻略,大战,大片,大片播放器,大航海时代4,大话西游,大话西游梦幻西游,大香蕉,大魔王,天刀,天命主宰,天堂,天天来塔防,天天魔域,天尊传奇游戏,天怒传奇,天气,天气预报,天气预报15天,天气预报中国天气,天涯明月刀,天涯明月刀手游官网下载,天珠,天珠图片及价格,天谕,天龙八部手游影门派,天龙八部网络游戏,太古星辰诀,头像女,头像男生,奇迹1.03h版,奇迹2,奇迹2中国公测,奇迹mu什么职业厉害,奇迹mu元素,奇迹mu单刷职业,奇迹mu大天使之剑,奇迹mu奇迹助手,奇迹mu小翅膀,奇迹mu战士武器大全,奇迹mu手机版,奇迹mu手游变态版,奇迹mu挂机,奇迹mu挂机点,奇迹mu王者归来,奇迹mu礼包领取,奇迹mu觉醒,奇迹mu觉醒手游,奇迹mu觉醒新服,奇迹sf,奇迹原版,奇迹变态版手游,奇迹奇迹,奇迹奇迹下载,奇迹成龙,奇迹手游,奇迹手游哪个职业厉害,奇迹手游天使武器,奇迹无双手游,奇迹最强者与mu,奇迹破坏神,奇迹第一版,奇迹面霜,奢侈品的包包,奥特曼2,奥特曼传奇英雄免内购,奥特曼格斗进化3,奥迪a4,奥迪a6l价格,奥迪q3,奥迪优惠,女人与,女人出轨后的5种反常表现,女人性,女帝,女朋友,女生,女生头像唯美,女生的游戏,好传奇,好玩的3d手游,好玩的三国策略游戏,好玩的仙侠手游推荐,好玩的传奇手游排行榜,好玩的类似传奇的手游,好看的动漫里番,好看的爱情动漫,好看视频,妲己,婚姻,孙悟空,孤岛惊魂,孤岛惊魂5,宁波,宅男福利,安卓手机管家,安卓游戏安卓,完结免费有声小说,完美世界免费阅读,完美世界游戏官网,官方麻将,宝骏,家常做法,家里装修,寂寞单身约会,寻找单身女人,封神单机版,封神召唤师,封魔传奇ol,射雕英雄传手游,将夜,小兵传奇,小冰冰传奇充值折扣,小冰冰传奇官网,小别墅,小孩,小小西游,小火锅,小米,小米版梦幻西游,小说,小说完结版玄幻,小说小说小说小说,小说排行,小说最新,小霸王,小鸡模拟器,少女前线,少帅你老婆又跑了小说,就是传奇,属羊,屠龙 传奇,屠龙之怒,屠龙传奇,屠龙冰雪手游,屠龙天下,屠龙无双,屠龙高爆,山海异兽录手游,山海经,山海经下载,山鸡传奇,崩坏3,巴塞罗那足球,帝国时代3亚洲王朝,帝国时代4,帝国时代之罗马帝国,帝国时代统治世界,帝王三国手机版,带切割的传奇手游,带狗传奇,带英雄无敌,幻想西游,幻灵手游,幽冥白虎,广州天气,广州火舞游戏决战沙邑,开奖结果,开局一把刀,开罗游戏,异世邪君,异兽,异兽鲲,异界之全职业大师,张大仙,张家辉代言的传奇手游,张家辉传奇,张家辉贪玩蓝月,张无忌,影院,征途2砸星,征途官方网,征途怀旧版,征途手机版官网,征途手游,征途新服,征途永恒,征途游戏下载,征途雷云风暴,很纯很暧昧,御魂,御龙在天手游,御龙天下,微信,微聚,忘仙,快手,快把我哥带走,快播电影在线观看网,快点阅读,快看小说,快红包,快视频快播,怀旧传奇,怀旧传奇游戏,怀旧版传奇手游,怀旧版本传奇,怒斩三国,怒砍一刀,性感女人内衣,性感少女,性感迷人美女,性感风骚美女图片,恐怖,情歌,情缘仙侠游戏,情话,成人用品无人售货机加盟,我叫mt,我想找女友,我想游戏中心,我本沉默传奇网站,我本沉默版本的传奇,我的世界1.4,我的战争,我秀直播,我秀美女直播,战争下载,战争指挥官,战争时期,战争电影,战争策略类单机游戏推荐,战歌,战歌龙城,战火与秩序,战狼传奇,战狼传奇网,战神之怒,战神归来,战神蚩尤,战舰世界,战觞奇迹,房价大跌,所有传奇,手抄报,手机,手机oppo,手机wifi,手机传奇,手机传奇游戏,手机华为,手机大型游戏排行榜,手机奇迹mu,手机屏幕,手机手机,手机游戏,手机游戏传奇,手机游戏养成,手机游戏排行榜前十名,手机游戏热血,手机版传奇,手机版传奇手游开服表,手机版单机传奇,手机版盛大传奇游戏,手机版集结号捕鱼游戏,手机直播软件哪个好,手机看片,手机网络游戏排行榜,手机迅雷,手机遥控麻将机,手游,手游中心,手游仙侠轻功,手游代理,手游传奇,手游传奇世界,手游传奇单机版无限元宝,手游传奇大全,手游传奇怀旧,手游传奇打金服网站,手游传奇挂机,手游传奇排行榜,手游传奇游戏平台,手游传奇神途,手游倩女幽魂,手游充值平台哪个好,手游刀塔传奇,手游卡牌三国,手游原始传奇礼包,手游号交易平台,手游大作,手游大全,手游大型,手游工作室,手游平台,手游手游,手游新,手游无限元宝,手游永恒纪元,手游热血传奇未知暗殿怎么走,手游版传奇排行第一,手游版奇迹,手游版奇迹mu,手游版热血传奇,手游版魔域,手游直播,手游破解版,手游网,手游赚人民币排行榜,手游飞车,手游魔兽世界,手游魔域,手游魔域有变态版吗,手表,扑鱼达人3,打boss,打传奇赚钱是真的吗,打怪手游,打渔游戏平台,打装备,打装备的手游,打金传奇,打金传奇单职业,打金传奇吧,打金传奇服,打金传奇游戏,打金手游,打鱼1000炮,打鱼怎么打,打鱼机,找个回合制手游,找传奇,找传奇的网站,找同城女人聊天,找离婚的女人,找老公的女人,技能,抖音号,抖音小游戏,抢红包,护肤,拇指玩,拳皇98,拳皇98终极之战,拳皇ol手游,拳皇三国,拳魂觉醒,挂机传奇手游,挂机打怪手游,捕鱼,捕鱼下载,捕鱼之海底捞无限金币,捕鱼之都捕鱼游戏平台,捕鱼乐翻天,捕鱼官方下载,捕鱼小游戏,捕鱼捕鱼捕,捕鱼棋牌下载,捕鱼棋牌游戏中心,捕鱼注册送金币,捕鱼游戏价格,捕鱼游戏千炮,捕鱼游戏捕鱼平台,捕鱼游戏无限金币无敌,捕鱼游戏能赚钱的,捕鱼破解版,捕鱼赢奖品,捕鱼达人千炮账号注册,捕鱼达人打鱼,捕鱼送6元,换三张麻将技巧,捷豹4s店,掀美女裙子,掌上dnf,排行榜小说,探探交友,推荐好玩的手游,撩妹聊天话题大全软件,操盘手炒股,擎天传奇,收购欧米茄手表,攻城掠地马超觉醒,攻速单职业,放开那三国2官网,放开那三国2最强阵容,放开那三国2礼包,放开那三国2阵容,放置传奇,放置手游,教主之家单职业,散人传奇平台,散人传奇手游单职业,散人天堂,斗小三的小说,斗破苍穹免费漫画,斗罗之你是我的人,斗罗四终极斗罗,斗罗大陆2之绝世唐门,斗罗大陆2漫画版,斗罗大陆3小说,斗罗大陆三龙王传说,斗罗大陆之,斗罗大陆之神界传说,斗罗大陆唐家三少,斗罗大陆小说下载,斗罗大陆小说免费阅读,斗罗大陆小说全文免费阅读,斗罗大陆手游官网,斗罗大陆武魂,斗罗大陆第,斗罗大陆绝世唐门漫画全集免费,斗罗大陆顶点小说,斗鱼tv,斩仙,斯诺克比赛,新仙剑传奇,新出的传奇手游,新复古传奇,新天龙八部公测,新开单职业传奇网,新征途,新征途游戏,新征途礼包,新房楼盘,新楼盘,新款本田crv价格,新版游戏,新的魔幻手游,新神魔大陆,新诛仙,新车报价,方舟生存进化,旅游攻略,无付费无vip传奇手游,无任务传奇手游,无尽裁决,无忧传奇官网,无敌升级,无敌小说,无敌版传奇,无限恐怖,无限打装备手游,日剧,日历,日韩,昆明新房,明末,易筋经,晚间天气预报,智商游戏,暗黑之路,暗黑传奇手游,暗黑传奇暗黑战,暗黑传奇礼包,暗黑使者,暗黑召唤师,暗黑复仇者,暗黑奇迹手游,暗黑无限,暗黑破坏神2和火炬之光2,暗黑破坏神3账号,暗黑系手游,暗黑者2,暗黑超神游戏,暴打三国志,暴风影音,最后的战争,最好玩的三国,最好的手游名字,最新天气天气预报,最新手游开服表,最新楼盘房价,最新款麻将机,最新的手游,最新的苹果手机,最火手游传奇,最火的传奇手游排行榜,最火角色扮演手游,月经,有手游传奇,有没有手游传奇,有跟传奇差不多的手游吗,服务器,木瓜奇迹手游官网,末世为王 小说,末日最强,末日求生游戏,本田凌派,本田雅阁优惠,杀神,李连杰,李逵劈鱼,李逵劈鱼打鱼,李逵捕鱼游戏下载,板寸头男生短发发型,极品狂少 我本疯狂 小说,林君河小说,林子聪代言传奇,林肯航海家,林肯飞行家,查找附近单身女人,格拉苏蒂手表,格斗版传奇,格斗类手游,桑蚕丝男士短袖t恤,梦三国手游官网,梦幻修仙网络游戏,梦幻图片,梦幻手游辅助,梦幻模拟战,梦幻神兽,梦幻神宠,梦幻花园,梦幻蜀山,梦幻西游2手游官网下载,梦幻西游之雷怒危机,梦幻西游单开稳定赚钱,梦幻西游哪个职业最厉害,梦幻西游坐骑成长,梦幻西游大,梦幻西游宝宝进阶,梦幻西游开区时间表,梦幻西游手戏,梦幻西游手游cdk兑换,梦幻西游手游下载官方,梦幻西游手游中文版,梦幻西游手游交易猫,梦幻西游手游什么时候公测,梦幻西游手游什么时候出新门派,梦幻西游手游代练,梦幻西游手游官方版下载,梦幻西游手游新手玩什么职业好,梦幻西游手游梦幻手游,梦幻西游手游武器外观,梦幻西游无双版新区,梦幻西游最贵的武器,梦幻西游服战号,梦幻西游网页版官网,梦幻西游要点卡吗,梦想世界,棋牌平台,棋牌捕鱼牛牛娱乐,棋牌麻将,植物大战,楚汉传奇电视剧免费观看,楚留香手游,楼盘价格,模拟器,横版格斗游戏,橡皮泥,欧米茄手表价格及图片,欧美一级特黄大片视频,欧美回合制游戏,歌曲,正版传奇世界手游,正版传奇手游,正版魔域官方网站,武侠之大宗师手游,武侠手游排行榜剑网三,武器,武林外传,武林外传手游变态版,武林群侠传,武破九荒,武神,武神至尊,武神风暴,武道帝尊,武道独尊,武道神尊,武道至尊 暗夜幽殇 小说,武道至尊全文免费阅读,死神手游,毒液传奇,比亚迪f3,比亚迪车型,毛河渡茅台镇原浆酒,毛片,毛笔行书书法,水库钓鱼,永恒之塔,永恒手游,永恒纪元手游,求小说,汽车,汽车报价下载,汽车轿车,沉底钓鱼饵料,沉默复古传奇,沉默版本传奇,沙巴克,沙巴克传奇,沙巴克传奇手机版,沙巴克传奇手游礼包,沙巴克攻城,沙盒游戏,洪荒之,浪琴,海洋捕鱼游戏,海贼王官方下载,海贼王官网,海贼王正版手游,海贼王白胡子,海贼王论坛,深圳,深夜福利直播平台,深渊主宰,混沌剑神最新章节,混沌弑神诀,清朝穿越小说,清风复古传奇,渡劫手游,渣渣辉传奇手游攻略,港,游戏,游戏game,游戏rpg,游戏三国,游戏充值,游戏制作,游戏复古传奇,游戏大厅游戏大厅,游戏排名,游戏排行榜,游戏玩的游戏,游戏直播,游戏能下载的,游戏陪玩,满贯捕鱼,漫威未来之战,漫威格斗,漫画,漫画航海王,激战2,火影,火影忍,火影忍者之鸣人,火影忍者剧场版,火影忍者忍者新世代,火影忍者手机游戏,火影忍者手游决,火影忍者漫画,火影格斗,火手游,火柴人联盟,火龙传奇,火龙复古传奇手游官网,火龙战神传奇,火龙版本传奇,灭绝丧尸屠城,灰姑娘,灵剑尊小说,灵魂传奇1.76金币版,炉石传奇,炉石传说任务,炉石传说盒子,炉石传说金币号,炉石传说隐藏任务,炮王捕鱼,烈火一刀,烈火传奇,烈火屠龙,烈火皇城,烈火雷霆,烈火龙城,烈焰,烈焰传奇,烈焰传奇手游官网,热火,热血1.76复古经典版本,热血176传奇,热血之刃下载,热血之怒,热血传奇,热血传奇1.76版官网,热血传奇2,热血传奇好玩吗,热血传奇开服,热血传奇手机版,热血传奇手游升级,热血传奇手游开服表,热血传奇手游道士攻略,热血传奇服务器列表,热血传奇端游,热血传奇装备列表,热血传奇论坛,热血传奇超变,热血传奇辅助,热血合击官网,热血战纪,热血江湖手游官网,热血群侠,热门传奇手游,热门电视剧,热门的游戏,焚天之怒下载,爆服版传奇游戏,爆率,爆率最高的传奇,爱你,爱动漫,爱奇艺动漫,爱彼手表,爱情恋爱,爱情网,爱趣游戏,爱阅读免费小说,爽翻捕鱼,牛牛,特种兵,特种兵在都市,狗狗,狙击手,狮子座,狼人杀游戏,猎魔人,玄幻奇幻小说排行榜完结,玄幻小说的巅峰,玄幻穿越小说排行榜完本,王城争霸,王座,王者争霸,王者代打,王者代练平台,王者传奇,王者传奇3,王者传奇手游官网,王者军团,王者出击,王者大陆,王者归来,王者荣耀一,王者荣耀之,王者荣耀之最强青铜,王者荣耀体验服申请,王者荣耀刷金币,王者荣耀后羿出装,王者荣耀廉颇,王者荣耀强势英雄,王者荣耀怎么充值,王者荣耀怎么充钱,王者荣耀无限刷点卷插件,王者荣耀最强王者,王者荣耀陪玩yy,王者荣耀雅典娜,玛法传奇,玛法英雄,玩传奇,玩吧游戏,玩的游戏游戏,现在什么传奇最火,现在手机,现在的传奇,现在还有传奇游戏吗,现在还有天龙八部荣耀服吗,琅琊榜,甄子丹,甄子丹代言传奇蓝月至尊版,甄子丹贪玩蓝月,生死狙击,用钱宝,电影大全免费观看,电影网,电玩城打鱼游戏下载,电玩城游戏大厅下载,电玩城游戏打鱼,电玩街机,电脑,电脑电脑,男人,男生,男穿越小说全本,畅游充值平台,留学条件,疫情,病毒,痛风,百度,百度输入法,百炼成神,的电影,皇城荣耀,皓月传奇,皓月版本传奇,皮带什么品牌的好,盒子,盗墓手游,盛世龙城,盛大传奇官网,盛大传奇手游,盛大传奇还有人玩吗,盛大热血传奇官方网,直播,直播秀色,看大片的播放器,看美女主播,真3d传奇手游,真正传奇手游,真武大帝,真魔幻手游,砍传奇,破剑之刃,破解版,破解版下载,社区,神与神,神兵传奇,神兽,神域,神奇宝贝xy,神奇宝贝口袋妖怪,神将传奇,神武,神武手游开服时间表,神武门派,神迹大陆,神途传奇,神途传奇官网,神途传奇客户端下载,神途传奇怎么样,神途传奇网站,神途传奇辅助,神途热血传奇,神都夜行录,福克斯,离异女人再婚,秋冬钓鲫鱼饵料,秘密,秦始皇,穿越火线新英雄武器,竖屏版的传奇游戏,策略国战手游,策略游戏,简单,篮球手游,米聊,米读小说,类传奇游戏,类似于传奇的手游,类似传奇的单机手游,类似传奇的手游,类似征途的游戏,类似梦幻西游的网页游戏,类似金庸群侠传武侠手游,类似魔兽的手游,精神分裂症,糖果传奇,红包,红包捕鱼游戏,红手指,红月至尊传奇手游,红烧肉的做法,红米,红警2坦克,红警3,红警坦克前线,红警手游,纯棉内裤,纸牌麻将,经典传奇bgm,经典传奇古墓,经典传奇古墓全集,经典传奇成龙,经典传奇手游,经典传奇手游排行榜,经典传奇闹鬼,经典奇迹,经典怀旧传奇,经典撩妹情话,经典版传奇,经典电视剧,经典老歌,经营,经营策略游戏,结婚的游戏,绝代双骄,绝地求生之刺激战场,绝版传奇,绿色传奇,绿茶龙井,网易新闻,网易购卡充值,网易阴阳师,网站,网络捕鱼游戏中心,网络棋牌,网络游戏传奇,网络游戏剑网3,网页传奇,网页版的梦幻西游,美女,美女 性感,美女三国游戏,美女主播在线观看,美女图片内衣,美女图片秀,美女壁纸,美女总裁的全能兵王萧晨,美女热舞直播,美女直播app,美女直播房间,美女福利直播,美女秀场,美女秀场直播下载,美女秀场直播间,美女美女直播,美女遭,美食,群英三国单机,老传奇手游,老传奇游戏,老传奇金币版,老梁观世界,老男人,耳机,股票开盘,胡,胡莱三国真徐晃,能赚钱的传奇,腾讯传奇手游最强装备,腾讯手机管家,腾讯手游征途,自动捕鱼,至尊,至尊棋牌,至尊神魔,至尊蓝月攻略,航母,航海王剧场版,航海王强者之路,航海王破解版,艺术涂料,艾希,苏宁易购,苏州天气,英雄传奇吧,英雄无敌,英雄杀,英雄榜,英雄版传奇手游,英雄联盟点券充值,英雄联盟王者荣耀,英魂之刃,英魂之刃手游,苹果,苹果9,苹果9什么时候上市,苹果se,苹果下载软件,苹果怎么用,苹果手机,苹果手机价格,苹果手机怎么办,苹果手机怎么激活,苹果游戏,苹果的手机,苹果笔记本,苹果苹果,范冰冰,茶叶,草鱼,荒野求生游戏,荣耀,荣耀v20pro,荣耀华为,荣耀官网,荣耀手机,荣耀王者下载,莽荒纪,菜谱,葫芦侠,蒱鱼游戏,蒸馒头,蓝月传奇1.76金币版,蓝月传奇传奇,蓝月传奇吧,蓝月战神套装,蓝月至尊游戏,蓝月至尊版2,蓝月至尊版官网版下载,蓝月至尊版手机版,蓝月至尊版激活码,薛仁贵传奇,蜀门,血族,街机游戏盒,街机网络版金蝉捕鱼,街机达人捕鱼,表劳力士,衬衫,装备,装备全靠打,西安,西游战记,要出发周边游,观赏鱼,视频在线,视频直播下载,觉醒,觉醒剑魂,言情|免费小说,言情小說,评书大全集,诛仙官方,诡秘之主,诸神之战,谁主三国,谷原希美,象棋,象棋残局,贪玩官方平台手游,贪玩蓝月,贪玩蓝月传奇官网,贪玩蓝月传奇手机版,贪玩蓝月手机版,贪玩蓝月手机版官网,贪玩蓝月真的rmb回收吗,贪玩蓝月至尊版,贪玩蓝月至尊版官网,贪玩蓝月装备rmb回收,购手机,赚钱传奇手游,赚钱的回合制手游,赛车游戏,赢捕鱼,赤月传奇2,赤月传奇3,赤月复古传奇,起亚k3,起点中文网,超bt传奇手游,超变传奇,超变传奇手游加速版,超变传奇手游单职业,超变传奇新服,超变传奇新服网,超变态天龙八部,超变攻速版传奇,超变网页版传奇,超声波捕鱼,超性感美女图片,超攻速传奇手游,超级变态手游上线满级,超级战兵,超级武神,越南美女,趣笔阁免费阅读小说,跑车,路虎卫士报价,轩辕传奇手游,轻钢别墅生产厂家,迈腾,这个游戏,这是个奇迹,远征,远征ol手游,连天气预报,连尚阅读免费阅读,迷你世界,迷失版本传奇网站,送vip传奇,送捕鱼,适合散人玩的传奇,适合长期玩的传奇,逆天传奇,逆战,逆水寒,逍遥传奇游戏,途游捕鱼大作战,速腾,造化之门,道士带狗传奇,那些电影,邪王嗜宠鬼医狂妃,部落,部落冲突修改器,部落冲突破解版免费下载,都市绝品邪少小说,酱香型酒,酷狗音乐,重庆天气,金币版1.70传奇,金庸小说全集免费阅读,金蝉捕鱼,金蟾捕鱼9900,金蟾捕鱼价格,金蟾捕鱼和捕鱼,钓钓鱼技巧,钓鱼,钓鱼人必看钓鱼网,钓鱼手竿,钓鱼方法大全,钓鱼杆价格,钓鱼鱼竿,钓鲤鱼,钢结构别墅,钻石,钻石等级,铁血传奇1.76,镇魔曲手游,问道,问道坐骑,问道官网首页,问道手机版,问道手游什么时候开服,问道手游官方正版,问道手游游戏中心,问道攻略,阴阳师,阴阳师官网,阴阳师首无,阿拉德之怒手游下载,阿瓦隆之王,阿紫,附近,附近人,附近单身女人,附近找女人,附近的人约会,附近租房,附魔,陆小凤传奇手游,陈一发,陌秀直播,陪我,雄霸传奇,雄霸天下传奇游戏,集成墙板厂家,雷克萨斯es,雷克萨斯es300h报价,雷神,雷神怒击,雷霆万钧手游,霸下传奇,青云传奇,青云诀2手游官网,青岛,韩国第一性感美女,韩服dnf手游吧,领礼包,领红包,风云传奇,高德地图,高爆率传奇,鬼服传奇霸业,魔兽rpg,魔兽世界70级怀旧服,魔兽世界充值优惠,魔兽世界快速升级,魔兽世界怀旧服金币,魔兽世界怎么充值,魔兽世界怎么刷声望,魔兽世界战网客户端,魔兽世界战网客户端下载,魔兽世界无情,魔兽世界游戏充值,魔兽世界猎人新手攻略,魔兽世界礼包,魔兽世界联盟,魔兽世界英雄,魔兽争霸战役,魔兽充值,魔兽剧情,魔兽历史,魔兽怀旧,魔兽战场主演,魔兽战场游戏,魔兽战役,魔兽手游,魔兽挂机无限金币版,魔兽攻略,魔兽来袭,魔兽王座,魔兽部落,魔力宝贝,魔域,魔域2.4终极版,魔域下载中心,魔域互通版下载,魔域变态版,魔域变态版手游,魔域口袋版,魔域口袋版攻略,魔域塔防,魔域官方正版手游,魔域官网下载,魔域官网手机版,魔域小说,魔域开服时间表,魔域归来充值,魔域手机版,魔域手游变态版,魔域手游开服,魔域手游最新版本,魔域文化,魔域新任务,魔域新服,魔域版,魔域聚宝阁,魔域觉醒,魔域觉醒好玩吗,魔幻类手游,魔法无敌,鱼丸深海捕鱼千炮版,鱼竿,鱼钓,鲫鱼饵料,麻将,麻将怎么打,麻将技巧,麻将最新,黄小说,黄片,黄色,黄金皓月传奇,黑暗之魂,黑暗光年,黑暗骑士2,鼎力三国十连抽,龙之传奇,龙之传奇手游,龙城战歌单机版,龙王传说,龙珠布罗利无限二次元,龙珠游戏,龙珠觉醒,龙珠超剧场,龙神传奇,龙神传说斗罗大陆3,龙腾传奇手游,龙魂至尊" if (x!=x)|(x=='') else x)
    
    plan_result['device'] = 1   # IOS
    plan_result['age'] = "2,3,4,5"
    plan_result['sex'] = 2    # 男性
    
    # 出价方式
    plan_result['transFrom'] = 1
    plan_result['payMode'] = 1
    plan_result['ocpcLevel'] = 2
    plan_result['isSkipStageOne'] = True
    
#     plan_result['optimizeDeepTrans'] = plan_result['optimizeDeepTrans'].apply(lambda x:np.random.choice([False,True]))
    plan_result['optimizeDeepTrans'] = plan_result['optimizeDeepTrans'].apply(lambda x: False)
    plan_result['transType'] = plan_result['optimizeDeepTrans'].apply(lambda x:26 if x==False else 4)
    plan_result['ocpcBid'] = plan_result['optimizeDeepTrans'].apply(lambda x:random.randint(3300.00, 3400.00) if x==False else random.randint(190.00, 200.00))
    plan_result['deepOcpcBid'] = plan_result['optimizeDeepTrans'].apply(lambda x:random.randint(5800.00, 6000.00) if x==True else np.nan)
    plan_result['useRoi'] = plan_result['optimizeDeepTrans'].apply(lambda x:True if x==False else np.nan)
    plan_result['roiRatio'] = plan_result['optimizeDeepTrans'].apply(lambda x:0.05 if x==False else np.nan)
    plan_result['deepTransType'] = plan_result['optimizeDeepTrans'].apply(lambda x:np.nan if x==False else 26)

#     plan_result['ocpcBid'] = plan_result['ocpcBid'].apply(lambda x: random.randint(3000.00, 3200.00))
#     plan_result['roiRatio'] = 0.05
#     plan_result['useRoi'] = True     # ROI系数
#     plan_result['deepOcpcBid'] = np.nan
#     plan_result['deepTransType'] = np.nan
    
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.loc[i, ['autoExpansion','education', 'customAge', 'keywords',
           'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
           'excludeTrans', 'region', 'device', 'age']].to_json()))
    plan_result['audience'] = ad_info
    plan_result.drop(['autoExpansion','education', 'customAge', 'keywords',
           'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
           'excludeTrans', 'region', 'device', 'age'], axis=1, inplace=True)
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.loc[i, [ 'transType', 'ocpcLevel',
           'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
           'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi']].to_json()))
    plan_result['ocpc'] = ad_info
    plan_result.drop([ 'transType', 'ocpcLevel',
           'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
           'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi'], axis=1, inplace=True)
    plan_result['schedule'] = plan_result['ocpc'].apply(lambda x: [
                                                                        {
                                                                            "weekDay": 3,
                                                                            "startHour": 5,
                                                                            "endHour": 10
                                                                        },
                                                                        {
                                                                            "weekDay": 4,
                                                                            "startHour": 5,
                                                                            "endHour": 10
                                                                        }
                                                                    ])
#     plan_result['bgtctltype'] = 2  # 加速投放
    # 创意
    document = get_document()
    plan_result['materialstyle'] = 113
    plan_result['brand'] = plan_result.apply(lambda x: '幸存挑战' ,axis=1)
#     plan_result['subtitle'] = '下载领取中国区专属福利！'
    plan_result['subtitle'] = '烧脑闯关，你能过几关？'
    plan_result['userPortrait'] = "https://feed-image.baidu.com/0/pic/1256783882_-2000210126_1902099700.jpg"
    plan_result['titles'] = plan_result['subtitle'].apply(lambda x: np.random.choice(document,3))
    
    plan_result.to_csv('./plan_result.csv', index=0)  # 保存创建日志
    return plan_result

In [21]:
def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
#     open_api_url_prefix = "https://ptom-pre.caohua.com/"
    open_api_url_prefix = "https://ptom.caohua.com/"
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 45
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('结束....')
    return rsp_data

In [22]:
if __name__ == '__main__':
    ad_account_id_group = np.array([11784, 11785, 11786, 11787, 11788])   # 计划账号
    plan_num = 5   # 计划数量
    plan_result = get_plan(plan_num, ad_account_id_group)
    print('计划数量', plan_result.shape[0])
    get_ad_create(plan_result)

匹配的素材ID [42240 42241 44743 44944 45299 45534 45752 45762 45764 45799 46109 46266
 46351 46973 47219 47266 47273]


ValueError: a must be greater than 0 unless no samples are taken